In [1]:
DEBUG = True
N_DEBUG = 100

In [2]:
import pandas as pd
import numpy as np
import transformers

In [3]:
test = pd.read_pickle("./dataset_test.pkl")
test = test # always load the full dataset! (np.random.shuffle(tokenized_sentences)). slice the actual hybrid_documents if debugging!


documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py




#from gpt2outputdataset.detector_radford import DetectorRadford
#from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorGuo]#,DetectorRadford,DetectorDetectGPT]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer
explainer_classes = [LIME_Explainer, ]

In [4]:
detector = DetectorGuo()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import re

In [6]:
import torch
import pickle

In [7]:
DEVICE = "cuda"
pattern = re.compile(r"<extra_id_\d+>")



model = "t5-small"
cache_dir="./.cache"
mask_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model, cache_dir=cache_dir).to(DEVICE)
mask_tokenizer = transformers.AutoTokenizer.from_pretrained(model, model_max_length=mask_model.config.n_positions, cache_dir=cache_dir)#.to(DEVICE)

In [8]:
text = documents.iloc[0]#"This is a simple test sentence."


In [9]:
def get_pertubed_text(text, n=1):
    prediction_original = detector.predict_label([text])[0]

    past_generations = []
    perturbed_text = text
  #  print(text)
    for _ in range(0,n):
        while True: # do while 
            # generate a perturbed sample
            tokens = text.split(' ')
            mask = np.zeros_like(tokens, dtype=bool)
            mask[np.random.randint(0, len(mask))] = 1 # TODO number of tokens to mask

            i = 0
            for ii, (m, token) in enumerate(zip(mask, tokens)):
                if m:
                    tokens[ii] = "<extra_id_{}>".format(i)
                    i+=1
            i-=1
            masked_text = ' '.join(tokens)
            stop_id = mask_tokenizer.encode(f"<extra_id_{i+1}>")[0]


            tok = mask_tokenizer(masked_text, return_tensors="pt", padding=True).to(DEVICE)
            outputs = mask_model.generate(**tok, max_length=150, do_sample=True, top_p=1, num_return_sequences=1, eos_token_id=stop_id,)# multinomial sampling
                                #                stopping_criteria=StoppingCriteriaList([self.stopping_criteria])) # no gains for short tests
            mt = mask_tokenizer.batch_decode(outputs, skip_special_tokens=False)

            fills = [x for x in re.split(r"<extra_id_\d*>", mt[0]) if x != "<pad>"]

            for i, (token, m) in enumerate(zip(tokens, mask)):
                if m:
                    tokens[i] = fills.pop(0).strip()
            perturbed_text = " ".join(tokens)

            # check if this is a valid and new perturbation
            if (perturbed_text == text) or (perturbed_text in past_generations):
                continue
            if detector.predict_label([perturbed_text])[0] != prediction_original:
                continue
            else:
                break
        past_generations.append(perturbed_text)
    return past_generations
# TODO detectors are not fully deterministic

In [10]:
N_REPLACE = 1
documents_and_perturbations = []

for document, gold_label in zip(documents,gold_labels):
    if gold_label == 1:
        continue
    # set seeds here so perturbed documents are the same regardless of slice for documents when debugging (and explanations don't have to be regenerated)
    np.random.seed(42)
    torch.manual_seed(42)
    documents_and_perturbations.append((document, gold_label, get_pertubed_text(document, 10)))



In [11]:
for original, gl, perturbations in documents_and_perturbations:
#    print(original)
    prediction_original = detector.predict_label([original])[0]
  #  print("...")
    assert len(perturbations) == len(set(perturbations)), "Duplicates in perturbations"
    for p in perturbations:
        assert detector.predict_label([p])[0] == prediction_original, "Labels don't match, do you set seed in the detector?"
        
 #       print(p)
 #   print("------------------------")
    

In [12]:
# TODO store documents_and_perturbations
documents_and_perturbations

[("Yes, buying REITs (Real Estate Investment Trusts) can be compared to investing in real estate. REITs are companies that own or finance income-generating real estate, such as apartments, shopping centers, office buildings, and warehouses. They are required to distribute at least 90% of their taxable income to shareholders in the form of dividends, which can make them a good source of passive income. When you buy shares in a REIT, you are essentially investing in a diversified portfolio of real estate assets. Just like with any investment, there are risks involved, including the risk that the value of the REIT's assets may decline or that the REIT may not generate sufficient income to pay dividends to shareholders. It's important to carefully consider these and other risks before making any investment decisions.",
  False,
  ["Yes, buying REITs (Real Estate Investment Trusts) can be compared to investing in real estate. REITs are companies that own or finance income-generating real es

In [13]:
for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for original, gl, perturbations in documents_and_perturbations:
            explainer.get_explanation_cached(original)
            for p in perturbations:
                explainer.get_explanation_cached(p)

regen Yes, buying REITs (Real Estate Investment Trusts) can be compared to investing in real estate. REITs are companies that own or finance income-generating real estate, such as apartments, shopping centers, office buildings, and warehouses. They are required to distribute at least 90% of their taxable income to shareholders in the form of dividends, which can make them a good source of passive income. When you buy shares in a REIT, you are essentially investing in a diversified portfolio of real estate assets. Just like with any investment, there are risks involved, including the risk that the value of the REIT's assets may decline or that the value of the REIT's assets may not generate sufficient income to pay dividends to shareholders. It's important to carefully consider these and other risks before making any investment decisions.
regen Yes, buying REITs (Real Estate Investment Trusts) can be compared to investing in real estate. REITs are companies that own or finance income-ge

In [ ]:
# generate all explanations
